In [ ]:
import boto3
import json
import os
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from urlParse import *

In [56]:
load_dotenv()

shopify_api_key = os.getenv("SHOPIFY_API_KEY")
shopify_api_token = os.getenv("SHOPIFY_API_TOKEN")
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
tenant_id = os.getenv("TENANT_ID")

In [ ]:
# Get access token
token_url = 'https://login.microsoftonline.com/' + tenant_id + '/oauth2/v2.0/token'
data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
    'scope': 'https://graph.microsoft.com/.default'
}
response = requests.post(token_url, data=data)
access_token = response.json()['access_token']

In [ ]:
# Set up API request headers
api_url = 'https://graph.microsoft.com/v1.0/users/b0704321-9c2e-4df6-be62-884879f7548d/messages'
headers = {
    'Authorization': 'Bearer ' + access_token,
    'Accept': 'application/json',
    'Prefer': 'outlook.body-content-type="text"'
}

In [ ]:
# Parse through each request url to find all emails 
parser = OutlookParser(headers, None, api_url)

q = Queue()

try:
    while q.getsize() <= 3500:
        q.enqueue(parser)
        parser = OutlookParser(headers, None, parser.getnext())
except:
    q.enqueue(parser)



In [ ]:
index = 1

while not q.isEmpty():
    current_parse = q.dequeue()
    for message in current_parse.getvalue():
        try:
            text = message['body']['content']
            soup = BeautifulSoup(text, 'html.parser')
            important_text = soup.find(class_ = "message-text-2").get_text(strip = True)
            with open(f'question/{index}.txt', 'w') as file:
                file.write(important_text)
        except:
            continue



In [ ]:
api_url = 'https://graph.microsoft.com/v1.0/users/b0704321-9c2e-4df6-be62-884879f7548d/mailFolders/sentItems/messages'

parser = OutlookParser(headers, None, api_url)

q = Queue()

try:
    while q.getsize() <= 190:
        q.enqueue(parser)
        parser = OutlookParser(headers, None, parser.getnext())
except:
    q.enqueue(parser)

In [ ]:
index = 1

while not q.isEmpty():
    current_parse = q.dequeue()
    for message in current_parse.getvalue():
        
        text = message['body']['content']
        soup = BeautifulSoup(text, 'html.parser')
        important_text = soup.find(_class = "gmail_quote").get_text(strip = True)
        with open(f'response/{index}.txt', 'w') as file:
            file.write(important_text)
        index += 1
    print(index)

Product Data

In [ ]:
import re

def get_next_page(response):
    try:
        matches = re.findall(r'<([^>]+)>; rel="([^"]+)"', response)
        for match in matches:
            if 'next' in match:
                return match[0]
            else:
                continue
    except:
        return None


In [ ]:
headers = {
        'X-Shopify-Access-Token': shopify_api_token,
        'Content-Type': 'application/json'
    }

url = "https://okayamadenim.myshopify.com/admin/api/2023-04/collections/260758241368/products.json"

response = requests.get(url = url, headers = headers)

all_data = []

while get_next_page(response.headers['link']):
    
    for product in response.json()['products']:

        all_data.append((product['id'], product['title']))

    response = requests.get(get_next_page(response.headers['link']), headers = headers)

In [ ]:
region = "ap-northeast-1"

dynamodb_client = boto3.client("dynamodb", region_name = region)

table_name = "productName"

for product in all_data:

    item = {
        'id': {'N': str(product[0])},
        'name': {'S': product[1]}
    }

    response = dynamodb_client.put_item(
        TableName = table_name,
        Item = item
    )

    print(response['ResponseMetadata']['HTTPStatusCode'])

In [ ]:
import shopify

session = shopify.Session(f"{shopify_api_key}:{shopify_api_token}@okayamadenim.myshopify.com", "2023-04", shopify_api_token)